In [1]:
library(Seurat)
library(ComplexHeatmap)
library(circlize)
library(RColorBrewer)
library(SCopeLoomR)
library(AUCell)
library(SCENIC)

Attaching SeuratObject

Le chargement a nécessité le package : grid

ComplexHeatmap version 2.10.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))


circlize version 0.4.15
CRAN page: https://cran.r-project.org/package=circlize
Github page: https://github.com/jokergoo/circlize
Documentation: https://jokergoo.github.io/circlize_book/book/

If you use it in published research, please cite:
Gu, Z. circlize implements and enhances circular visualizat

In [2]:
getwd()

[1] "/work/adufour/notebook/scenic/visualisation_resultats/HeatMap"

In [3]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [4]:
setwd("/home/adufour/work/scenic/scenic_pig/direct_annot_v6/out")

In [5]:
load("/home/adufour/work/rds_storage/SCENIC/scenic_heatmap.RData")

In [6]:
cluster_info <- read.csv(file = '/home/adufour/work/notebook/scenic/preparation/cell_annotation/cell_type_annotation.csv')

In [7]:
colnames(cluster_info) <- c("index", "cell", "cluster")

In [8]:
time <- read.csv(file = '/home/adufour/work/notebook/scenic/preparation/cell_annotation/time.csv')

In [9]:
colnames(time) <- c("index", "cell", "time")

In [10]:
cellInfo <- merge(cluster_info,time,by="cell")

In [11]:
rownames(cellInfo) <- cellInfo$cell

In [12]:
cellInfo$time_type <-paste(cellInfo$time,cellInfo$cluster,sep="_")

In [13]:
head(cellInfo)

,cell,index.x,cluster,index.y,time,time_type
,<chr>,<int>,<chr>,<int>,<chr>,<chr>
AAACCCAAGACGCTCC-1_3_3-8,AAACCCAAGACGCTCC-1_3_3-8,9722,HYPO Ma,8019,J9,J9_HYPO Ma
AAACCCAAGACTGGGT-1_1_4-10,AAACCCAAGACTGGGT-1_1_4-10,22133,HYPO Ma,18237,J11,J11_HYPO Ma
AAACCCAAGCAACAGC-1_1_4-10,AAACCCAAGCAACAGC-1_1_4-10,18157,TE Ma,18238,J11,J11_TE Ma
AAACCCAAGCCGTTGC-1_4_3-9,AAACCCAAGCCGTTGC-1_4_3-9,14538,HYPO Ve,10880,J9,J9_HYPO Ve
AAACCCAAGGCCCACT-1_2_4-11,AAACCCAAGGCCCACT-1_2_4-11,27514,TE Gr,27711,J11,J11_TE Gr
AAACCCAAGGTGCTAG-1_3_3-8,AAACCCAAGGTGCTAG-1_3_3-8,9723,HYPO Ma,8020,J9,J9_HYPO Ma


In [14]:
regulons_unfiltered <- regulons

In [15]:
regulons_filtered <- readRDS("/home/adufour/work/rds_storage/SCENIC/regulons_filterby_rss_tissue_annot.rds")

In [16]:
regulons_te <- regulons[unique(regulons_filtered[regulons_filtered$variable %in% c("TE Ma", "TE Gr", "TE Mu", "TE In", "TE Ea", "TE Lr", "TE Il1", "TE Mt"),]$regulons)]

In [17]:
# Split the cells by cluster
cellsPerTimeCluster <- split(rownames(cellInfo), cellInfo[,"cluster"])
regulonAUC_te <- regulonAUC[onlyNonDuplicatedExtended(names(regulons_te)),]
# Calculate average expression:
                                     
regulonActivity_byCellTypeTime <- sapply(cellsPerTimeCluster,
                                     function(cells) rowMeans(getAUC(regulonAUC_te)[,cells]))

In [18]:
liste_a <- c("CEBPB (+)", "SREBF1 (+)", "E2F4 (+)", "CEBPG (+)", "RELA (+)", "CTCF (+)", "TCF12 (+)", "MAZ (+)", "MYC (+)", "E2F6 (+)", "E2F1 (+)", "ZNF143 (+)",
             "STAT1 (+)", "GABPA (+)", "BCLAF1 (+)", "SP1 (+)", "SREBF2 (+)", "MAFF (+)", "YY1 (+)", "ATF6 (+)", "FOSB (+)", "XBP1 (+)", "THAP1 (+)", "ETV4 (+)",
             "FOXA2 (+)", "RCOR1 (+)", "NFYA (+)", "REST (+)", "KLF17 (+)")

liste_b <- c("MAFG (+)", "GFI1B (+)", "ZNF148 (+)", "KLF15 (+)", "HOXA7 (+)", "BACH2 (+)", "NEUROD2 (+)", "SMARCC1 (+)", "ETV3 (+)", "ZNF281 (+)", "MEOX1 (+)",
             "HOXB6 (+)", "ENSSSCG00000021322 (+)", "CLOCK (+)", "CEBPB (+)", "SREBF1 (+)", "E2F4 (+)", "NR1H4 (+)", "FOSL1 (+)", "NFYC (+)", "ZBTB33 (+)",
             "TLX3 (+)", "KLF9 (+)", "HOXA5 (+)", "RXRG (+)", "MAX (+)", "CEBPG (+)", "TBR1 (+)", "TLX2 (+)", "ZIC5 (+)", "HMX2 (+)", "TBX3 (+)", "POU2F2 (+)",
             "LHX2 (+)", "GATA1 (+)", "RELA (+)", "CTCF (+)", "RUNX3 (+)", "E2F8 (+)", "SETDB1 (+)", "KLF8 (+)", "EGR2 (+)", "MAFK (+)", "BARHL1 (+)", "TCF12 (+)",
             "ENSSSCG00000017516 (+)", "ATF1 (+)", "RFX5 (+)", "MAZ (+)", "BARHL2 (+)", "ENSSSCG00000028997 (+)", "KLF3 (+)", "HOXB8 (+)", "TCF7 (+)", "EGR1 (+)",
             "DLX3 (+)", "MBD2 (+)", "HNF4G (+)", "KLF2 (+)", "MITF (+)", "FOS (+)", "EVX1 (+)", "PATZ1 (+)", "PPARG (+)", "ARNT (+)", "SP3 (+)", "MYC (+)",
             "RORC (+)", "ELK4 (+)", "ZFP42 (+)", "ETS2 (+)", "LEF1 (+)", "HESX1 (+)", "ARNT2 (+)", "ZNF471 (+)", "ZNF454 (+)", "UNCX (+)", "E2F6 (+)", "PBX3 (+)",
             "CREB1 (+)", "E2F1 (+)", "EMX1 (+)", "ZNF143 (+)", "STAT1 (+)", "RFX6 (+)", "CFP (+)", "VEZF1 (+)", "ELK3 (+)", "ELF4 (+)", "BACH1 (+)", "CPEB1 (+)",
             "ARID3A (+)", "IRF3 (+)", "GABPA (+)", "KLF4 (+)", "ESRRB (+)", "KLF6 (+)", "PPARD (+)", "IKZF1 (+)", "BCLAF1 (+)", "IRF1 (+)", "MSX1 (+)", "PAX5 (+)",
             "HOXD3 (+)", "SOX5 (+)", "HOXA2 (+)", "MNT (+)", "ZIC2 (+)", "E2F7 (+)", "SP1 (+)", "HIF1A (+)", "FOXN2 (+)", "IRF8 (+)", "OTX2 (+)", "SREBF2 (+)",
             "IRF7 (+)", "MAFF (+)", "YY1 (+)", "BHLHE40 (+)", "ATF6 (+)", "FOSB (+)", "LHX1 (+)", "HOXB7 (+)", "ATF3 (+)", "DLX1 (+)", "RORB (+)", "FOXO4 (+)",
             "MIXL1 (+)", "SEBOX (+)", "HOXB5 (+)", "XBP1 (+)", "TAL1 (+)", "SMAD5 (+)", "THAP1 (+)", "NR2C1 (+)", "HOXA10 (+)", "KLF5 (+)", "ETV4 (+)", "KLF12 (+)",
             "ETV1 (+)", "DLX2 (+)", "FOXA2 (+)", "FOSL2 (+)", "NRF1 (+)", "ENSSSCG00000000278 (+)", "ESR1 (+)", "SP4 (+)", "HOXA1 (+)", "RCOR1 (+)", "NR2C2 (+)",
             "PAX6 (+)", "HOXB4 (+)", "IKZF2 (+)", "HEY1 (+)", "STAT3 (+)", "ETS1 (+)", "CREB5 (+)", "NFYA (+)", "ELF2 (+)", "NFYB (+)", "PBX1 (+)", "ELF1 (+)",
             "REST (+)", "KLF17 (+)", "KLF13 (+)", "KLF10 (+)", "HOXB3 (+)", "TFAP2C (+)", "CREB3 (+)", "KLF7 (+)", "GABPB1 (+)", "ESR2 (+)")

liste_c <- c("CEBPB (+)", "HES6 (+)", "SREBF1 (+)", "E2F4 (+)", "CEBPG (+)", "NANOG (+)", "RELA (+)", "CTCF (+)", "GATA4 (+)", "CEBPZ (+)", "SRF (+)", "TCF12 (+)",
             "NR2F6 (+)", "NFKB2 (+)", "MAZ (+)", "MYC (+)", "KLF11 (+)", "E2F6 (+)", "E2F1 (+)", "ZNF143 (+)", "STAT1 (+)", "GABPA (+)", "BCLAF1 (+)", "KLF16 (+)",
             "SP1 (+)", "SREBF2 (+)", "MAFF (+)", "YY1 (+)", "ATF6 (+)", "FOSB (+)", "ZBTB7A (+)", "XBP1 (+)", "THAP1 (+)", "ETV4 (+)", "FOXA2 (+)", "GATA3 (+)",
             "ESRRG (+)", "RCOR1 (+)", "TEAD2 (+)", "NFYA (+)", "REST (+)", "KLF17 (+)")

classif <- function (x) {
    if (x %in% liste_a) {
        v <- "1"
        }
    else if (x %in% liste_b) {
        v <- "2"
        }
    else if (x %in% liste_c) {
        v <- "3"
        }
    else {
        v <- NA
        }
    return (v)
    }

In [19]:
pch <- lapply(names(regulons_te), classif)

In [22]:
mat <- scale(t(regulonActivity_byCellTypeTime[,c("TE Ma", "TE Gr", "TE Mu", "TE In", "TE Ea", "TE Lr", "TE Il1", "TE Mt")]), center = T, scale=T)

In [45]:
library(svglite)

In [60]:
options(repr.plot.width=25, repr.plot.height=8) # To set the figure size in Jupyter
colnames(mat) <- gsub(x = colnames(mat), pattern = "(+)", replacement = "", fixed = TRUE)
ha <- HeatmapAnnotation(Regulons_size = anno_barplot(lengths(regulons_te), height = unit(20, "mm")), annotation_name_gp = grid::gpar(fontsize=20), annotation_label = "Regulons size")
hb <- HeatmapAnnotation(pub = anno_simple(rep(NA,dim(mat)[2]), na_col = "white", pch = pch), show_annotation_name = FALSE)
col_fun <- colorRamp2(seq(min(mat), max(mat), length = 100), colorRampPalette(rev(brewer.pal(n = 7, name = "RdYlBu")))(100))
lgd <- list(title = "Regulon activity", legend_height = unit(6, "cm"), grid_width = unit(1, "cm"), labels_gp = gpar(fontsize = 20), title_gp = gpar(fontsize = 18))
svglite('/home/adufour/work/notebook/plots/scenic/regulon_expression_te.svg', width = 25, height = 8)
hm <- draw(ComplexHeatmap::Heatmap(mat, col = col_fun, rect_gp = gpar(col = "grey", lwd = 1), row_names_gp = grid::gpar(fontsize=24),
                                   column_names_gp = grid::gpar(fontsize=16, fontface='italic'), top_annotation = ha, bottom_annotation = hb,
                                   heatmap_legend_param = lgd), padding = unit(c(2, 2, 2, 2), "mm"))
dev.off()

png 
  2

In [72]:
regulons_hypo <- regulons[unique(regulons_filtered[regulons_filtered$variable %in% c("HYPO Ma", "HYPO Ve", "HYPO In", "HYPO Ea"),]$regulons)]

In [73]:
# Split the cells by cluster
cellsPerTimeCluster <- split(rownames(cellInfo), cellInfo[,"cluster"])
regulonAUC_hypo <- regulonAUC[onlyNonDuplicatedExtended(names(regulons_hypo)),]
# Calculate average expression:
                                     
regulonActivity_byCellTypeTime <- sapply(cellsPerTimeCluster,
                                     function(cells) rowMeans(getAUC(regulonAUC_hypo)[,cells]))

In [74]:
mat <- scale(t(regulonActivity_byCellTypeTime[,c("HYPO Ma", "HYPO Ve", "HYPO In", "HYPO Ea")]), center = T, scale=T)

In [75]:
pch <- lapply(names(regulons_hypo), classif)

In [76]:
options(repr.plot.width=25, repr.plot.height=8) # To set the figure size in Jupyter
colnames(mat) <- gsub(x = colnames(mat), pattern = "(+)", replacement = "", fixed = TRUE)
ha <- HeatmapAnnotation(Regulons_size = anno_barplot(lengths(regulons_hypo), height = unit(20, "mm")), annotation_name_gp = grid::gpar(fontsize=20), annotation_label = "Regulons size")
hb <- HeatmapAnnotation(pub = anno_simple(rep(NA,dim(mat)[2]), na_col = "white", pch = pch), show_annotation_name = FALSE)
col_fun <- colorRamp2(seq(min(mat), max(mat), length = 100), colorRampPalette(rev(brewer.pal(n = 7, name = "RdYlBu")))(100))
lgd <- list(title = "Regulon activity", legend_height = unit(6, "cm"), grid_width = unit(1, "cm"), labels_gp = gpar(fontsize = 20), title_gp = gpar(fontsize = 18))
svglite('/home/adufour/work/notebook/plots/scenic/regulon_expression_hypo.svg', width = 18, height = 7)
hm <- draw(ComplexHeatmap::Heatmap(mat, col = col_fun, rect_gp = gpar(col = "grey", lwd = 1), row_names_gp = grid::gpar(fontsize=24),
                                   column_names_gp = grid::gpar(fontsize=16, fontface='italic'), top_annotation = ha, bottom_annotation = hb,
                                   heatmap_legend_param = lgd), padding = unit(c(2, 2, 2, 2), "mm"))
dev.off()

png 
  2

In [77]:
regulons_epi <- regulons[unique(regulons_filtered[regulons_filtered$variable %in% c("ICM_E5", "EPI_E7", "EPI_E9", "EPI_E11"),]$regulons)]

In [78]:
# Split the cells by cluster
cellsPerTimeCluster <- split(rownames(cellInfo), cellInfo[,"cluster"])
regulonAUC_epi <- regulonAUC[onlyNonDuplicatedExtended(names(regulons_epi)),]
# Calculate average expression:
                                     
regulonActivity_byCellTypeTime <- sapply(cellsPerTimeCluster,
                                     function(cells) rowMeans(getAUC(regulonAUC_epi)[,cells]))

In [79]:
mat <- scale(t(regulonActivity_byCellTypeTime[,c("ICM_E5", "EPI_E7", "EPI_E9", "EPI_E11")]), center = T, scale=T)

In [80]:
pch <- lapply(names(regulons_epi), classif)

In [81]:
options(repr.plot.width=25, repr.plot.height=8) # To set the figure size in Jupyter
colnames(mat) <- gsub(x = colnames(mat), pattern = "(+)", replacement = "", fixed = TRUE)
ha <- HeatmapAnnotation(Regulons_size = anno_barplot(lengths(regulons_epi), height = unit(20, "mm")), annotation_name_gp = grid::gpar(fontsize=20), annotation_label = "Regulons size")
hb <- HeatmapAnnotation(pub = anno_simple(rep(NA,dim(mat)[2]), na_col = "white", pch = pch), show_annotation_name = FALSE)
col_fun <- colorRamp2(seq(min(mat), max(mat), length = 100), colorRampPalette(rev(brewer.pal(n = 7, name = "RdYlBu")))(100))
lgd <- list(title = "Regulon activity", legend_height = unit(6, "cm"), grid_width = unit(1, "cm"), labels_gp = gpar(fontsize = 20), title_gp = gpar(fontsize = 18))
row_order <- rev(c("ICM_E5", "EPI_E7", "EPI_E9", "EPI_E11"))
svglite('/home/adufour/work/notebook/plots/scenic/regulon_expression_epiblast.svg', width = 17, height = 7)
hm <- draw(ComplexHeatmap::Heatmap(mat, col = col_fun, rect_gp = gpar(col = "grey", lwd = 1), row_names_gp = grid::gpar(fontsize=24),
                                   column_names_gp = grid::gpar(fontsize=16, fontface='italic'), top_annotation = ha, row_order = row_order, bottom_annotation = hb,
                                   heatmap_legend_param = lgd), padding = unit(c(2, 2, 2, 2), "mm"))
dev.off()

png 
  2